In [1]:
import re
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

##### Check connection to OpenAI 

In [2]:
client = OpenAI()

In [3]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)
chat_completion.choices[0].message.content

'Hello! How are you today?'

### Define Agent class

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

### Define ReAct prompt

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_italian_salary:
e.g. average_dog_weight: lawyer
returns the average Italian salary when a job title is given

Example session:

Question: How much does a dentist earn in Italy?
Thought: I should look the average italian salary using average_italian_salary
Action: average_dog_weight: dentist
PAUSE

You will be called again with this:

Observation: A dentist earns on average 80,000 euros per year in Italy

You then output:

Answer: A dentist earns 80,000 euros per year in Italy
""".strip()

prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_italian_salary:\ne.g. average_dog_weight: lawyer\nreturns the average Italian salary when a job title is given\n\nExample session:\n\nQuestion: How much does a dentist earn in Italy?\nThought: I should look the average italian salary using average_italian_salary\nAction: average_dog_weight: dentist\nPAUSE\n\nYou will be called again with this:\n\nObservation: A dentist earns on average 80,000 euros per year in Italy\n\nYou then output:\n\nAnswer: A dentist earns 80,000 euros per year i

### Define Actions

In [6]:
def calculate(what):
    return eval(what)

def average_italian_salary(job):
    if job in "lawyer": 
        return("a lawyer earns on average 80,000 euros")
    elif job in "architect":
        return("an architect earns on average 40,000 euros")
    elif job in "software engineer":
        return("a software engineer earns on average 50,000 euros")
    else:
        return("a worker in Italy earns on average 30,000 a year")

known_actions = {
    "calculate": calculate,
    "average_italian_salary": average_italian_salary
}

### Step by step reasoning

#### Example 1

In [7]:
ag = Agent(prompt)
result = ag("How much does an architect earn in Italy?")

In [8]:
print(result)

Thought: I should look up the average Italian salary for an architect using the available action.
Action: average_italian_salary: architect
PAUSE


In [9]:
result = average_italian_salary("architect")
result

'an architect earns on average 40,000 euros'

In [10]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: an architect earns on average 40,000 euros'

In [11]:
ag.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_italian_salary:\ne.g. average_dog_weight: lawyer\nreturns the average Italian salary when a job title is given\n\nExample session:\n\nQuestion: How much does a dentist earn in Italy?\nThought: I should look the average italian salary using average_italian_salary\nAction: average_dog_weight: dentist\nPAUSE\n\nYou will be called again with this:\n\nObservation: A dentist earns on average 80,000 euros per year in Italy\n\nYou then output:\n\nAnswer: A dent

In [12]:
ag(next_prompt)

'Answer: An architect earns on average 40,000 euros per year in Italy.'

In [13]:
ag.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_italian_salary:\ne.g. average_dog_weight: lawyer\nreturns the average Italian salary when a job title is given\n\nExample session:\n\nQuestion: How much does a dentist earn in Italy?\nThought: I should look the average italian salary using average_italian_salary\nAction: average_dog_weight: dentist\nPAUSE\n\nYou will be called again with this:\n\nObservation: A dentist earns on average 80,000 euros per year in Italy\n\nYou then output:\n\nAnswer: A dent

#### Example 2

In [14]:
ag = Agent(prompt)
question = """An engaged couple is an architect and a lawyer. On average, how much do they earn together?"""
result = ag(question)

In [15]:
print(result)

Thought: I need to find the average Italian salary for both an architect and a lawyer, then sum these amounts to determine their combined earnings.
Action: average_italian_salary: architect
PAUSE


In [16]:
next_prompt = "Observation: {}".format(average_italian_salary("architect"))
print(next_prompt)

Observation: an architect earns on average 40,000 euros


In [17]:
result = ag(next_prompt)
print(result)

Thought: Now I need to find the average Italian salary for a lawyer.
Action: average_italian_salary: lawyer
PAUSE


In [18]:
next_prompt = "Observation: {}".format(average_italian_salary("lawyer"))
print(next_prompt)

Observation: a lawyer earns on average 80,000 euros


In [19]:
result = ag(next_prompt)
print(result)

Thought: I now have the average salaries for both an architect (40,000 euros) and a lawyer (80,000 euros). I need to sum these amounts to find their combined earnings.
Action: calculate: 40000 + 80000
PAUSE


In [20]:
next_prompt = "Observation: {}".format(eval("40000 + 80000"))
print(next_prompt)

Observation: 120000


In [21]:
result = ag(next_prompt)
print(result)

Answer: Together, an architect and a lawyer earn on average 120,000 euros per year in Italy.


### Implementation with a loop

In [22]:
action_re = re.compile('^Action: (\w+): (.*)$')

In [23]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [24]:
question = """An engaged couple is an architect and a lawyer. On average, how much do they earn together?"""
query(question)

Thought: I need to find the average salaries for both an architect and a lawyer in Italy and then sum them up to get their combined earnings.
Action: average_italian_salary: architect
PAUSE
 -- running average_italian_salary architect
Observation: an architect earns on average 40,000 euros
Thought: Now I need to find the average salary for a lawyer in Italy.
Action: average_italian_salary: lawyer
PAUSE
 -- running average_italian_salary lawyer
Observation: a lawyer earns on average 80,000 euros
Thought: I now have the average salaries for both an architect (40,000 euros) and a lawyer (80,000 euros). I need to sum these amounts to find their combined earnings.
Action: calculate: 40000 + 80000
PAUSE
 -- running calculate 40000 + 80000
Observation: 120000
Answer: An engaged couple consisting of an architect and a lawyer earns on average 120,000 euros per year together in Italy.
